# 載入套件及資料集

In [36]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Layers for FNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

# Layers for CNN
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD, Adam

# For data preprocessing
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# 資料前處理-使用fashion mnist資料集

In [37]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [38]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [39]:
x_train = x_train.reshape(60000,28,28,1)/255
x_test = x_test.reshape(10000,28,28,1)/255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 打造神經網路模型

In [40]:
CNN_layers = [Conv2D(16, (3, 3), input_shape=(28, 28, 1), padding='same', activation='relu', name='Conv_1'),
              MaxPool2D(),
              Conv2D(32, (3, 3), padding='same', activation='relu', name='Conv_2'),
              MaxPool2D(),
              Conv2D(64, (3, 3), padding='same', activation='relu', name='Conv_3'),
              GlobalAveragePooling2D()]

FC_layers = [Dense(units=256, activation='relu'),
             Dense(units=10, activation='softmax')]

In [41]:
CNN_layers + FC_layers

In [42]:
model = Sequential(CNN_layers+FC_layers)
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv_1 (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
Conv_2 (Conv2D)              (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
Conv_3 (Conv2D)              (None, 7, 7, 64)          18496     
_________________________________________________________________
global_average_pooling2d_4 ( (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)              

In [43]:
model.compile(loss='categorical_crossentropy', 
                optimizer=Adam(),
                metrics=['categorical_accuracy'])

In [44]:
model.fit(x_train, y_train, batch_size=128, epochs=10,validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 38s 635us/sample - loss: 1.0396 - categorical_accuracy: 0.6138 - val_loss: 0.7415 - val_categorical_accuracy: 0.7296
Epoch 2/10
60000/60000 [==============================] - 39s 649us/sample - loss: 0.6961 - categorical_accuracy: 0.7381 - val_loss: 0.6693 - val_categorical_accuracy: 0.7520
Epoch 3/10
60000/60000 [==============================] - 43s 721us/sample - loss: 0.6223 - categorical_accuracy: 0.7654 - val_loss: 0.6094 - val_categorical_accuracy: 0.7703
Epoch 4/10
60000/60000 [==============================] - 47s 783us/sample - loss: 0.5635 - categorical_accuracy: 0.7889 - val_loss: 0.5685 - val_categorical_accuracy: 0.7956
Epoch 5/10
60000/60000 [==============================] - 49s 818us/sample - loss: 0.5239 - categorical_accuracy: 0.8072 - val_loss: 0.5168 - val_categorical_accuracy: 0.8136
Epoch 6/10
60000/60000 [==============================] - 49s 819us/sample 

In [45]:
score_train = model.evaluate(x_train, y_train)
score_test = model.evaluate(x_test, y_test)

print(f'Train Accuracy: {score_train[1]*100}')
print(f'Test Accuracy: {score_test[1]*100}')

10000/10000 [==============================] - 2s 219us/sample - loss: 0.4125 - categorical_accuracy: 0.8550
Train Accuracy: 86.56333088874817
Test Accuracy: 85.50000190734863


# 以下做遷移學習：mnist資料集

In [46]:
(u_train, v_train), (u_test, v_test) = mnist.load_data()

In [47]:
u_train = u_train.reshape(60000,28,28,1)/255
u_test = u_test.reshape(10000,28,28,1)/255
v_train = to_categorical(v_train, 10)
v_test = to_categorical(v_test, 10)

In [48]:
FC_layers_2 = [Dense(units=54, activation='relu'),
                   Dense(units=10, activation='softmax')]

In [49]:
model_2 = Sequential(CNN_layers + FC_layers_2)
model_2.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv_1 (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
Conv_2 (Conv2D)              (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
Conv_3 (Conv2D)              (None, 7, 7, 64)          18496     
_________________________________________________________________
global_average_pooling2d_4 ( (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 54)               

# Frozen:凍結借來的部分，只針對新建立的神經網路層訓練

In [50]:
for layer in CNN_layers:
    layer.trainable = False

In [51]:
model_2.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv_1 (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
Conv_2 (Conv2D)              (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
Conv_3 (Conv2D)              (None, 7, 7, 64)          18496     
_________________________________________________________________
global_average_pooling2d_4 ( (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 54)               

In [52]:
model_2.compile(loss='categorical_crossentropy', 
                optimizer=Adam(),
                metrics=['categorical_accuracy'])

In [54]:
model_2.fit(u_train, v_train, batch_size=128, epochs=10,validation_data=(u_test, v_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 12s 198us/sample - loss: 1.5858 - categorical_accuracy: 0.5134 - val_loss: 0.9978 - val_categorical_accuracy: 0.7391
Epoch 2/10
60000/60000 [==============================] - 11s 180us/sample - loss: 0.7867 - categorical_accuracy: 0.7925 - val_loss: 0.6197 - val_categorical_accuracy: 0.8242
Epoch 3/10
60000/60000 [==============================] - 14s 241us/sample - loss: 0.5558 - categorical_accuracy: 0.8444 - val_loss: 0.4716 - val_categorical_accuracy: 0.8691
Epoch 4/10
60000/60000 [==============================] - 12s 196us/sample - loss: 0.4574 - categorical_accuracy: 0.8679 - val_loss: 0.4080 - val_categorical_accuracy: 0.8848
Epoch 5/10
60000/60000 [==============================] - 12s 206us/sample - loss: 0.4034 - categorical_accuracy: 0.8814 - val_loss: 0.3660 - val_categorical_accuracy: 0.8959
Epoch 6/10
60000/60000 [==============================] - 12s 202us/sample 

In [57]:
score_train = model_2.evaluate(u_train, v_train)
score_test = model_2.evaluate(u_test, v_test)

print(f'Train Accuracy: {score_train[1]*100}')
print(f'Test Accuracy: {score_test[1]*100}')

10000/10000 [==============================] - 2s 229us/sample - loss: 0.2795 - categorical_accuracy: 0.9152
Train Accuracy: 91.07999801635742
Test Accuracy: 91.51999950408936
